<a href="https://colab.research.google.com/github/ridhimagoyal/Object-Detection-Basic/blob/main/Pulling_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-kresgkx5
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-kresgkx5
  Resolved https://github.com/tweepy/tweepy.git to commit db28c0e84826485755eb7fcef0c30f75395dff5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
pip install tweepy


In [9]:
# import tweepy
# import pandas as pd

# # Replace with your Bearer Token
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAFrt2QEAAAAANy2XWP%2B9xTUaeCPNsbu3AbpQi%2Bg%3DScPVtPd5u4fEh5xwKRoQREg26fsmP2kFsNOoTsFkXWzMeBbddG"

# # Initialize Tweepy Client
# client = tweepy.Client(bearer_token=bearer_token)

# # Define your keywords
# keywords = ["block", "fight", "strike"]
# query = " OR ".join(keywords) + " -is:retweet -is:reply"  # Exclude retweets and replies

# # Fetch recent tweets
# response = client.search_recent_tweets(
#     query=query,
#     tweet_fields=["created_at", "public_metrics", "author_id", "lang"],
#     max_results=50,  # Max per request is 100
# )

# # Parse results into a DataFrame (if tweets found)
# if response.data:
#     tweets_data = [
#         {
#             "Tweet": tweet.text,
#             "Created At": tweet.created_at,
#             "Author ID": tweet.author_id,
#             "Like Count": tweet.public_metrics["like_count"],
#             "Retweet Count": tweet.public_metrics["retweet_count"],
#             "Language": tweet.lang
#         }
#         for tweet in response.data
#     ]
#     df = pd.DataFrame(tweets_data)
#     print(df)
# else:
#     print("No tweets found matching your keywords.")


In [14]:
import tweepy
import pandas as pd
from textblob import TextBlob

bearer_token = "AAAAAAAAAAAAAAAAAAAAABL62QEAAAAATjpXxoWAhG8GlSasvchAovUdnV0%3D4kF0Jqm8UpUb2EA5u843upHk9LWFGH5mHRwvtFdQ7Ur9B0CmTE"

client = tweepy.Client(bearer_token=bearer_token)

keywords = [
    "protest", "strike", "shutdown", "roadblock", "blockade",
    "rail roko", "andolan", "dharna", "rasta roko", "traffic jam"
]

target_regions = ["Delhi", "Mumbai","Hyderabad"]
query = "(" + " OR ".join(keywords) + ")" + " (" + " OR ".join(target_regions) + ")" + " -is:retweet -is:reply"

response = client.search_recent_tweets(
    query=query,
    tweet_fields=["created_at", "public_metrics", "author_id", "lang"],
    max_results=50,
)

def classify_tweet(text):
    text_lower = text.lower()
    if "protest" in text_lower or "andolan" in text_lower:
        event_type = "Protest"
    elif "strike" in text_lower or "dharna" in text_lower:
        event_type = "Strike"
    elif "shutdown" in text_lower:
        event_type = "Shutdown"
    elif any(word in text_lower for word in ["traffic jam", "roadblock", "rasta roko"]):
        event_type = "Blockage"
    else:
        event_type = "Other"

    # Severity analysis using sentiment
    polarity = TextBlob(text).sentiment.polarity
    severity = "High" if polarity < -0.3 else "Medium" if polarity < 0 else "Low"

    return event_type, severity

# Store tweets with classification
tweets_data = []
if response.data:
    for tweet in response.data:
        event_type, severity = classify_tweet(tweet.text)
        tweets_data.append({
            "Tweet": tweet.text,
            "Created At": tweet.created_at,
            "Author ID": tweet.author_id,
            "Like Count": tweet.public_metrics["like_count"],
            "Retweet Count": tweet.public_metrics["retweet_count"],
            "Language": tweet.lang,
            "Event Type": event_type,
            "Severity": severity
        })

    df = pd.DataFrame(tweets_data)
    print(df)
else:
    print("No relevant tweets found.")


                                                Tweet  \
0   #AirIndia: पटना से गाजियाबाद पहुंची एयर इंडिया...   
1   Same Action Needed first in Hyderabad and foll...   
2   India’s silence on Israel’s strike inside Iran...   
3   Surrender was what UPA did in 2008 after terro...   
4   UNBELIEVABLE:\nOn 04th August 2023, Air India ...   
5   #Mumbai ki #Baarish…\nkabhi #Traffic jam, kabh...   
6   मुंबईत पुन्हा मराठीची गळचेपी, टिळा पुसण्यास नक...   
7   https://t.co/ogbY7Jqf1I\nIf you protest and al...   
8   DPS Dwarka fee row continues, parents protest ...   
9   DPS Dwarka fee row continues, parents protest ...   
10  Dwarka ppl shud enjoy the BJp rule ! \nThey vo...   
11  DPS Dwarka fee row continues, parents protest ...   
12  Such a horrendous spree of tragedies this year...   
13  1970s :: Girls of Delhi University Taking Out ...   
14  22 June - Dharna at JANTAR MANTAR, DELHI \n\nR...   
15  Delhi: Parents protest against DPS Dwarka fee ...   
16  Time for M/s. @netanyahu &a

In [11]:
supplier_regions = {
    "SupplierA": "Delhi",
    "SupplierB": "Mumbai",
    "SupplierC": "Hyderabad"
}
severity_score = {
    "High": 3,
    "Medium": 2,
    "Low": 1
}

# Assuming `df` is your DataFrame from the Twitter fetch
def calculate_risk(df, supplier_regions):
    severity_score = {"High": 3, "Medium": 2, "Low": 1}
    region_scores = {region: 0 for region in set(supplier_regions.values())}

    for _, row in df.iterrows():
        for region in region_scores:
            if region.lower() in row["Tweet"].lower():
                region_scores[region] += severity_score[row["Severity"]]

    supplier_risk = {}
    for supplier, region in supplier_regions.items():
        score = region_scores.get(region, 0)
        if score >= 9:
            risk = "High Risk"
        elif score >= 4:
            risk = "Medium Risk"
        else:
            risk = "Low Risk"
        supplier_risk[supplier] = {"Region": region, "Score": score, "Risk Level": risk}

    return supplier_risk


In [19]:
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk


nltk.download('vader_lexicon')

data = [
    {"Tweet": "Major protest happening at India Gate, Delhi! Had a huge impact on the citizens, many died and many were injured.", "Created At": "2025-06-09 14:30", "Location": "Delhi"},
    {"Tweet": "Heavy traffic due to roadblock in Mumbai suburbs", "Created At": "2025-06-10 09:00", "Location": "Mumbai"},
    {"Tweet": "Calm day in Hyderabad with no disruptions.", "Created At": "2025-06-08 16:00", "Location": "Hyderabad"},
    {"Tweet": "Andolan continues near Delhi border", "Created At": "2025-06-11 11:45", "Location": "Delhi"},
    {"Tweet": "Rail roko protest blocks trains in Chennai.", "Created At": "2025-06-07 13:15", "Location": "Chennai"},
    {"Tweet": "Dharna against water issues in Hyderabad city center", "Created At": "2025-06-10 18:20", "Location": "Hyderabad"},
]

df = pd.DataFrame(data)
df.head()
df["Created At"] = pd.to_datetime(df["Created At"])


keywords = ["protest", "strike", "shutdown", "roadblock", "blockade", "rail roko", "andolan", "dharna", "rasta roko", "traffic jam"]
search_locations = ["Delhi", "Mumbai", "Hyderabad", "Chennai"]
cutoff_date = datetime.now() - timedelta(days=3)

def filter_tweet(text):
    return any(kw in text.lower() for kw in keywords)

filtered_df = df[
    (df["Location"].isin(search_locations)) &
    # (df["Created At"] >= cutoff_date) &
    (df["Tweet"].apply(filter_tweet))
].copy()

vader = SentimentIntensityAnalyzer()


def analyze_sentiments(text):
    blob = TextBlob(text)
    vader_scores = vader.polarity_scores(text)
    return pd.Series({
        "TextBlob_Polarity": blob.sentiment.polarity,
        "TextBlob_Subjectivity": blob.sentiment.subjectivity,
        "VADER_Compound": vader_scores["compound"],
        "VADER_Positive": vader_scores["pos"],
        "VADER_Negative": vader_scores["neg"],
        "VADER_Neutral": vader_scores["neu"]
    })

sentiment_results = filtered_df["Tweet"].apply(analyze_sentiments)
result_df = pd.concat([filtered_df, sentiment_results], axis=1)


print(result_df)


                                               Tweet          Created At  \
0  Major protest happening at India Gate, Delhi! ... 2025-06-09 14:30:00   
1   Heavy traffic due to roadblock in Mumbai suburbs 2025-06-10 09:00:00   
3                Andolan continues near Delhi border 2025-06-11 11:45:00   
4        Rail roko protest blocks trains in Chennai. 2025-06-07 13:15:00   
5  Dharna against water issues in Hyderabad city ... 2025-06-10 18:20:00   

    Location  TextBlob_Polarity  TextBlob_Subjectivity  VADER_Compound  \
0      Delhi           0.369531                 0.6000         -0.7424   
1     Mumbai          -0.162500                 0.4375          0.0000   
3      Delhi           0.100000                 0.4000          0.0000   
4    Chennai           0.000000                 0.0000         -0.4404   
5  Hyderabad          -0.100000                 0.1000          0.0000   

   VADER_Positive  VADER_Negative  VADER_Neutral  
0           0.089           0.332          0.57

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from google.colab import files
tweets_df.to_csv('tweets.csv')
files.download('tweets.csv')